## 1. Import Python Libraries

In [ ]:
import os, sagemaker, subprocess, boto3, time

## 2. (OPTIONAL) Download Images from S3

In [ ]:
bucket_name = "uniben-data"
prefix = "yolo-validation/"   
local_dir = "images-test"

if not os.path.isdir(local_dir):
    raise FileNotFoundError(f"Local directory not found: {local_dir}")
        
s3 = boto3.client('s3')
paginator = s3.get_paginator('list_objects_v2')

for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
    for obj in page.get('Contents', []):
        key = obj['Key']
        if key.endswith('/'):
            continue  # Skip Folders
        filename   = os.path.basename(key)
        local_path = os.path.join(local_dir, filename)
        print('.', end='')
        s3.download_file(bucket_name, key, local_path)
print("\nDone downloading images!")

## 3. TRAININGS

### 3.1. Install Ultralytics for YOLO11 model

https://docs.ultralytics.com/models/yolo11/#supported-tasks-and-modes

In [ ]:
!pip3 install ultralytics
from ultralytics import YOLO

## Choose a base model:
model_name = 'yolo11x.pt'
model = YOLO(model_name)

### 3.2. Train the model

In [ ]:
print("Start time:", time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
train_start_time = time.time()

model.train(data="uniben-label.yaml", 
            epochs=1000, 
            imgsz=720, 
            augment=True, 
            batch=10, 
            verbose=False) # Toggle the epoch summaries

train_end_time = time.time()
print(f"Training Time = {(train_end_time - train_start_time)/60:0.2f} minutes.")

### 3.3. (OPTIONAL) Resume training from the lastest epoch

In [ ]:
## Choose a model:
train = "train2"
print("Latest train:", train)

In [ ]:
model = YOLO(f'runs/detect/{train}/weights/last.pt')
model.train(resume=True)

## 4. Zip the code and model into `model.tar.gz` and upload it to specific S3 bucket
Here permission is granted to the S3 bucket created with CDK and not any other bucket

In [13]:
model_zip = "best.pt"
code_zip = "code/"
bashCommand = f"tar -cpzf  model.tar.gz {zip_model} {code_zip}"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

## 5. Select yolo11 bucket

In [ ]:
s3_client = boto3.client('s3')
response = s3_client.list_buckets()
for bucket in response['Buckets']:
    if 'yolo11' in bucket["Name"]:
        bucket = 's3://' + bucket["Name"]
        break
print(f'Bucket: {bucket}')

## 6. Upload model to S3

In [ ]:
from sagemaker import s3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
print(f'Account ID: {account_id}')
role = get_execution_role()
print(f'Role: {role}')

prefix = "yolo11"
model_data = s3.S3Uploader.upload("model.tar.gz", bucket + "/" + prefix)
print(f'Model Data: {model_data}')

## 7. Create the SageMaker PyTorchModel

In [16]:
from sagemaker.pytorch import PyTorchModel

sess = sagemaker.Session(default_bucket=bucket.split('s3://')[-1])
model = PyTorchModel(entry_point='inference.py',
                     model_data=model_data, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLO11_MODEL': 'best.pt'},
                     sagemaker_session=sess)

## 8. Deploy the model on SageMaker Endpoint:

In [ ]:
from sagemaker.deserializers import JSONDeserializer

INSTANCE_TYPE = 'ml.c5.xlarge'
ENDPOINT_NAME = 'yolo11-pytorch-' + str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

# Store the endpoint name in the history to be accessed by 2_TestEndpoint.ipynb notebook
%store ENDPOINT_NAME
print(f'Endpoint Name: {ENDPOINT_NAME}')

predictor = model.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         deserializer=JSONDeserializer(),
                         endpoint_name=ENDPOINT_NAME)